### 1. Load data

I'll be using a voting dataset I made.

---

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import cross_val_score

from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, ElasticNetCV
from sklearn.linear_model import Lasso, Ridge, ElasticNet

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor

%matplotlib inline

In [3]:
issues = pd.read_csv('./datasets/city_issues_nlp.csv')

### 2. Create X, Y, and normalize X

**X is num_votes here**. We're doing regression.

---

In [4]:
Y = issues.num_votes.values

X = issues[[x for x in issues.columns if x not in ['num_votes']]]
X_cols = X.columns
X = (X - X.mean())/X.std()
X = X.values

### 3. Linear regression

1. Cross-validate a linear regression with 5 folds. 
2. Build a linear regression model with the full X and Y.

---

In [6]:
linreg = LinearRegression()
linreg_score = np.mean(cross_val_score(linreg, X, Y, cv=5))
linreg.fit(X,Y)
print linreg_score

-2.74117855525e+27


### 4. Ridge regression

1. Either use `GridSearchCV` or `RidgeCV` to find the best `C` or `alpha` respectively. **Remember that bigger alphas means stronger regularization, and smaller Cs are stronger regularization. (C is the inverse of alpha).**
2. Cross-validate the R2 with Ridge using your optimal C or alpha.
3. Build a final Ridge model and fit it on the full X and Y as you did above.

---

In [10]:
ridge_params = {
    'alpha':np.logspace(0,4,50)
}

ridge_gs = GridSearchCV(Ridge(), ridge_params, cv=5, verbose=1, n_jobs=-1)
ridge_gs.fit(X, Y)

ridge = ridge_gs.best_estimator_
ridge_score = np.mean(cross_val_score(ridge, X, Y, cv=5))
ridge.fit(X,Y)
print ridge_score

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:    9.4s finished


0.276137927598


### 5. Lasso regression

1. Use either `GridSearchCV` or `LassoCV` to find the optimal `C` or `alpha` for the Lasso regression.
2. Cross-validate the R2 with Lasso using your optimal C or alpha.
3. Build a final Lasso model fit on the full X and Y.

---

In [12]:
lasso_params = {
    'alpha':np.logspace(-2,2,200)
}

lasso_gs = GridSearchCV(Lasso(), lasso_params, cv=5, verbose=1, n_jobs=-1)
lasso_gs.fit(X, Y)

lasso = lasso_gs.best_estimator_
lasso_score = np.mean(cross_val_score(lasso, X, Y, cv=5))
lasso.fit(X,Y)
print lasso_score

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    9.0s
[Parallel(n_jobs=-1)]: Done 646 tasks      | elapsed:   19.4s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   26.7s finished


0.316287188052


### 6. ElasticNet regression

Now you'll get to try out the ElasticNet. It is a combination of the Ridge and the Lasso to leverage the benefits of both!

Arguments to optimize:

    alpha : same as the Ridge/Lasso above
    l1_ratio: this is the proportion of Ridge vs Lasso that the model is. 
        An l1_ratio of 0.0 is a pure Ridge
        An l1_ratio of 1.0 is a pure Lasso
        
1. Use `GridSearchCV` or `ElasticNetCV` to search for the optimal `alpha` and `l1_ratio`. 
2. Explain the probable reason why the it chose the parameters it did as the best ones.
3. Cross-validate the R2 with the ElasticNet using the optimal parameters.
4. Fit the ElasticNet on all X and Y.

---

In [13]:
enet_cv = ElasticNetCV(l1_ratio=np.linspace(0.01,1.0,25),
                       n_alphas=100, cv=5, n_jobs=-1)
enet_cv.fit(X,Y)

enet = ElasticNet(l1_ratio=enet_cv.l1_ratio_, alpha=enet_cv.alpha_)
enet_score = np.mean(cross_val_score(enet, X, Y, cv=5))
enet.fit(X,Y)
print enet_score

0.316278012198


//anaconda/lib/python2.7/site-packages/sklearn/linear_model/coordinate_descent.py:466: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations
  ConvergenceWarning)


### 7. DecisionTreeRegressor

1. Use `GridSearchCV` to find the best `max_features`, `max_depth`, and `min_samples_leaf`. Read the documentation and think about what range of parameters would be good to search for each!
2. Cross-validate the R2 as above.
3. Fit a DecisionTreeRegressor on all X and Y as above.

---

In [16]:
dtr = DecisionTreeRegressor()

dtr_params = {
    'max_features':[None],
    'max_depth':[1,2,3,None],
    'min_samples_leaf':np.linspace(1,101,5)
}

dtr_gs = GridSearchCV(dtr, dtr_params, cv=5)
dtr_gs.fit(X,Y)

dtr = dtr_gs.best_estimator_
dtr_score = np.mean(cross_val_score(dtr, X, Y, cv=5))
dtr.fit(X,Y)
print dtr_score

0.350793681653


### 8. BaggingRegressor

Now we'll use bagging with the DecisionTreeRegressor. Yes, regressions can be done with bagging too!

---

Remember that with the Bagging regressor you first have to initialize the internal "base estimator" that it will copy:

```python
dtr = DecisionTreeRegressor()
```

A cool thing to note is that you can actually gridseach over the internal base estimators as well. So, not only are you finding the best parameters for the BaggingRegressor but also the DecisionTreeRegressors that it copies inside:

```python
bag_params = {
    'base_estimator__max_features':[None],
    'base_estimator__max_depth':[None],
    'base_estimator__min_samples_leaf':[1],
    'max_features': [0.33, 0.66, 0.99],
    'max_samples': [0.1, 0.2, 0.4, 0.6, 0.8, 0.9],
    'n_estimators': [100]
}
```

**Be careful putting too many parameters into the `GridSearchCV`! It can really explode the possible permutations!**

That being said, you'll probably be able to put a decent amount of parameters in since the wine dataset doesn't have many columns.

Next you initialize the BaggingRegressor, putting the desired model as the first argument:

```python
bag = BaggingRegressor(dtr)
```

This tells the BaggingRegressor that you want it to spawn DecisionTreeRegressors as it's internal "children" base estimators.

Lastly, you'll put the BaggingClassifier into the grid searcher and fit on the data (it will cross-validate with the specified `cv` folds.

```python
bag_gs = GridSearchCV(bag, bag_params, cv=5, verbose=1)
bag_gs.fit(X, Y)
```

---

As before...

1. Use `GridSearchCV` to find the best `BaggingRegressor` and optionally internal `DecisionTreeRegressor` parameters.
2. Cross-validate the R2.
3. Fit a `BaggingRegressor` on all X and Y with the optimal parameters.

---

In [17]:
dtr = DecisionTreeRegressor()

bag_params = {
    'base_estimator__max_features':[None],
    'base_estimator__max_depth':[None],
    'base_estimator__min_samples_leaf':[1],
    'max_features': [0.33, 0.66, 0.99],
    'max_samples': [0.1, 0.2, 0.4, 0.6, 0.8, 0.9],
    'n_estimators': [100]
}

bag = BaggingRegressor(dtr)

bag_gs = GridSearchCV(bag, bag_params, cv=5, verbose=1, n_jobs=-1)
bag_gs.fit(X,Y)

bag = bag_gs.best_estimator_
bag_score = np.mean(cross_val_score(bag, X, Y, cv=5))
bag.fit(X,Y)
print bag_score


Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   34.6s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:  2.0min finished


0.406788148278


### 9. Get feature importances from `RandomForestRegressor`

The `RandomForestRegressor` has an attribute called `.feature_importances_`. These are the importances of your predictors as measured by how useful they were to the base estimators.

As you may recall, the `RandomForestRegressor` is just a special case of the `BaggingRegressor` that specifically uses decision trees. In fact, you've already done it above. The difference is that this class gives us the feature importances whereas the "generalized" bagging regressor class does not.

1. Save the column names X in a variable.
2. Grid search optimal parameters for the `RandomForestRegressor`.
3. Fit a `RandomForestRegressor` using the optimal parameters you found on the full X and Y.
4. Get out the feature importances.
5. Create a pandas DataFrame where one column is the feature importances and the other column is the X column names.
6. Sort the dataframe you made by feature importances in descending order.
7. Plot the feature importances.

---

In [19]:
rf_params = {
    'max_features':['auto',None],
    'max_depth':[1,2,3,None],
    'min_samples_leaf':np.linspace(1,101,5),
    'n_estimators':[250]
}

rf = RandomForestRegressor()
rf_gs = GridSearchCV(rf, rf_params, cv=5, verbose=1, n_jobs=-1)
rf_gs.fit(X, Y)

rf = rf_gs.best_estimator_
rf_score = np.mean(cross_val_score(rf, X, Y, cv=5))
rf.fit(X, Y)
print rf_score

feat_imp = rf.feature_importances_
feature_importances = pd.DataFrame({'feature':X_cols, 'importance':feat_imp})
feature_importances.sort_values(['importance'], ascending=False)

print feature_importances.head()


Fitting 5 folds for each of 40 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   55.6s


KeyboardInterrupt: 

In [ ]:
sns.set(style="whitegrid")

f, ax = plt.subplots(figsize=(6, 15))

sns.set_color_codes("muted")
sns.barplot(x="importance", y="feature", data=feature_importances,
            label="feature importances", color="b")

sns.despine(left=True, bottom=True)

plt.show()

### 10. [BONUS] Use a different regression class with the `BaggingRegressor`

You could try `Ridge`, `Lasso`, `ElasticNet`, `SVC`, `KNeighborsRegressor` or any kind of regression class you're interested in!

---